In [1]:
import pandas as pd
import numpy as np
import pickle
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Create address column 

In [2]:
#load address data 
address_data = pd.read_csv('../data/sc122a.txt', delimiter = '\t', header = 0, encoding='latin-1')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,9,22,23,26,28,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
street = address_data['LSTREE']
city = address_data['LCITY']
zipcode = list(map(str, address_data['LZIP'])) #convert all int in LZIP to strings so they can be appended 
state = address_data['LSTATE']

In [4]:
def make_address(street, city, state, zipcode): 
    '''Returns a new list of complete addresses with combined street, city, state, and zipcode 
    '''
    addresses = [] #empty list to fill with full addresses
    for i in np.arange(len(street)): 
        full_address = street[i].lower() + ' ' + city[i].lower() + ' ' + state[i].lower() + ' ' + zipcode[i].lower() #for every street in list of streets, append city, state, and zip to create full address
        addresses.append(full_address) #append full address into empty list of addresses
    return addresses 

In [5]:
full_address = make_address(street, city, state, zipcode) #list of full addresses 

In [6]:
address_data['ADDRESS13'] = full_address #add column of full addresses

In [7]:
#select only NCESSCH and ADRESS13 columns because we're joining on NCESSCH and we only want to add the ADDRESS13 values to charters_2015.pkl
address_NCESSCH_data = address_data[['NCESSCH', 'ADDRESS13']] 

## Match formatting of addresses and names in CER dataframe to formatting of new address data 

In [8]:
CER = pd.read_csv('CER_2012-13.csv', encoding = 'latin-1') #open CER file 

In [9]:
#loop through every address, removing commas and periods, and converting all letters to lowercase
for i in np.arange(len(CER['CER_ADDRESS'])):
    CER.loc[i,'CER_ADDRESS'] = CER.loc[i,'CER_ADDRESS'].replace(',', '').replace('.','').lower() 


In [10]:
#uniform formatting of name by making all characters lowercase and removing all punctuation
CER['CER_NAME'] = [str(name).lower() for name in CER['CER_NAME']]

## Add address column to nonduplicate charter dataframe

In [11]:
#load nonduplicate charter data 
charters_nonduplicate = pd.read_csv('../../nowdata/parsing/overlaps_removed_df.csv', sep = '\t', header = 0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (128,129,130,131,132,133,134,135,137,196,197,198,211,299,300,321,335) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#merge addresses to charters_nonduplicate based on NCESSCH 
charters_nonduplicate_with_address = pd.merge(charters_nonduplicate, address_NCESSCH_data, how = 'left', on = 'NCESSCH') 

In [13]:
#match formatting for name and address 
charters_nonduplicate_with_address['SCHNAM12'] = [str(name).lower() for name in charters_nonduplicate_with_address['SCHNAM12']]

charters_nonduplicate_with_address['ADDRESS13'] = [str(address) for address in charters_nonduplicate_with_address['ADDRESS13']]

## Create new column of combined street+address to use as comparison with fuzzy ratio

In [14]:
#combine name and address for charters 
chartername = charters_nonduplicate_with_address['SCHNAM12']
charteraddress = charters_nonduplicate_with_address['ADDRESS13']
charters_nonduplicate_with_address['address_name_combined'] = [name+' '+address for name,address in zip(chartername, charteraddress)]
#charters_nonduplicate_with_address
                                                               
#combine name and address for CER 
CERname = CER['CER_NAME']
CERaddress = CER['CER_ADDRESS']
CER['address_name_combined'] = [name+' '+address for name,address in zip(CERname, CERaddress)]

## Using fuzzy ratio 

1. List fuzzy ratio between name+address columns in both data frames 
2. Keep the highest fuzzy ratio 
3. If fuzzy ratio is over 70, change the column name to its match 

In [15]:

index = 0
for combined_charters in charters_nonduplicate_with_address['address_name_combined']:
    print(index)
    ratio = [fuzz.ratio(combined_charters, combinedCER) for combinedCER in CER['address_name_combined']]
    greatest_match_index = np.asarray(ratio).argmax()
    greatest_match_name = ratio[greatest_match_index]
    if greatest_match_name >= 70:
        print('name match found')
        charters_nonduplicate_with_address.loc[index, 'address_name_combined'] = CER.loc[greatest_match_index, 'address_name_combined']
    index = index + 1
        

0
1
name match found
2
name match found
3
name match found
4
name match found
5
name match found
6
7
8
name match found
9
name match found
10
name match found
11
name match found
12
13
name match found
14
15
name match found
16
name match found
17
name match found
18
name match found
19
name match found
20
name match found
21
name match found
22
23
name match found
24
25
name match found
26
name match found
27
28
name match found
29
name match found
30
31
name match found
32
name match found
33
name match found
34
35
36
name match found
37
name match found
38
name match found
39
name match found
40
name match found
41
name match found
42
name match found
43
44
45
46
name match found
47
48
name match found
49
50
51
52
name match found
53
name match found
54
name match found
55
name match found
56
57
name match found
58
59
name match found
60
61
62
63
64
65
name match found
66
name match found
67
name match found
68
69
70
71
72
73
74
75
76
77
name match found
78
name match found
79
80
81

name match found
646
name match found
647
name match found
648
name match found
649
name match found
650
651
652
name match found
653
name match found
654
name match found
655
name match found
656
657
658
659
660
name match found
661
name match found
662
name match found
663
664
665
name match found
666
667
name match found
668
name match found
669
name match found
670
name match found
671
name match found
672
673
674
675
name match found
676
677
678
679
680
name match found
681
name match found
682
name match found
683
name match found
684
name match found
685
name match found
686
name match found
687
688
689
name match found
690
name match found
691
692
693
name match found
694
695
name match found
696
name match found
697
name match found
698
699
name match found
700
701
name match found
702
703
704
705
706
name match found
707
name match found
708
name match found
709
name match found
710
name match found
711
name match found
712
name match found
713
name match found
714
715
name m

name match found
1282
name match found
1283
name match found
1284
name match found
1285
name match found
1286
name match found
1287
name match found
1288
name match found
1289
name match found
1290
name match found
1291
name match found
1292
name match found
1293
name match found
1294
name match found
1295
name match found
1296
name match found
1297
name match found
1298
name match found
1299
name match found
1300
1301
name match found
1302
name match found
1303
name match found
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
name match found
1326
1327
1328
1329
name match found
1330
1331
name match found
1332
name match found
1333
name match found
1334
name match found
1335
1336
1337
name match found
1338
1339
1340
1341
name match found
1342
name match found
1343
1344
1345
name match found
1346
1347
1348
1349
name match found
1350
1351
1352
name match found
1353
1354
name match found
1355
name match found
1356
name match fo

name match found
1813
name match found
1814
name match found
1815
name match found
1816
name match found
1817
1818
1819
name match found
1820
1821
1822
1823
name match found
1824
name match found
1825
name match found
1826
name match found
1827
name match found
1828
name match found
1829
name match found
1830
name match found
1831
name match found
1832
1833
name match found
1834
name match found
1835
1836
name match found
1837
name match found
1838
name match found
1839
name match found
1840
name match found
1841
name match found
1842
name match found
1843
name match found
1844
name match found
1845
1846
1847
name match found
1848
name match found
1849
name match found
1850
name match found
1851
name match found
1852
name match found
1853
name match found
1854
name match found
1855
1856
name match found
1857
name match found
1858
name match found
1859
name match found
1860
1861
name match found
1862
name match found
1863
name match found
1864
name match found
1865
name match found
1866

name match found
2345
2346
2347
2348
name match found
2349
name match found
2350
name match found
2351
2352
name match found
2353
2354
name match found
2355
name match found
2356
name match found
2357
2358
name match found
2359
name match found
2360
name match found
2361
2362
name match found
2363
name match found
2364
name match found
2365
2366
name match found
2367
name match found
2368
name match found
2369
name match found
2370
name match found
2371
2372
name match found
2373
name match found
2374
name match found
2375
name match found
2376
2377
name match found
2378
name match found
2379
name match found
2380
name match found
2381
2382
name match found
2383
name match found
2384
name match found
2385
name match found
2386
2387
2388
name match found
2389
name match found
2390
name match found
2391
name match found
2392
name match found
2393
name match found
2394
name match found
2395
name match found
2396
name match found
2397
name match found
2398
name match found
2399
2400
name m

...

name match found
8127
name match found
8128
name match found
8129
name match found
8130
name match found
8131
name match found
8132
name match found
8133
8134
8135
name match found
8136
name match found
8137
name match found
8138
8139
name match found
8140
name match found
8141
8142
name match found
8143
name match found
8144
name match found
8145
name match found
8146
name match found
8147
name match found
8148
name match found
8149
name match found
8150
8151
8152
name match found
8153
name match found
8154
name match found
8155
name match found
8156
name match found
8157
name match found
8158
name match found
8159
name match found
8160
name match found
8161
name match found
8162
8163
name match found
8164
name match found
8165
name match found
8166
name match found
8167
name match found
8168
name match found
8169
name match found
8170
name match found
8171
name match found
8172
8173
name match found
8174
name match found
8175
name match found
8176
name match found
8177
8178
name matc

name match found
8684
name match found
8685
name match found
8686
name match found
8687
name match found
8688
name match found
8689
name match found
8690
name match found
8691
name match found
8692
name match found
8693
name match found
8694
8695
name match found
8696
name match found
8697
name match found
8698
name match found
8699
name match found
8700
name match found
8701
name match found
8702
name match found
8703
name match found
8704
name match found
8705
name match found
8706
name match found
8707
name match found
8708
name match found
8709
8710
name match found
8711
name match found
8712
name match found
8713
name match found
8714
name match found
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728
8729
8730
8731
8732
8733
8734
8735
8736
8737
8738
name match found
8739
name match found
8740
8741
8742
name match found
8743
8744
8745
name match found
8746
8747
8748
8749
name match found
8750
name match found
8751
name match found
8752
8753
8754
8755
8756
8757
8

9378
9379
9380
9381
9382
9383
9384
9385
9386
9387
name match found
9388
9389
9390
9391
9392
9393
name match found
9394
9395
9396
9397
name match found
9398
9399
name match found
9400
name match found
9401
name match found
9402
9403
name match found
9404
9405
name match found
9406
9407
name match found
9408
name match found
9409
name match found
9410
9411
name match found
9412
9413
name match found
9414
name match found
9415
9416
9417
name match found
9418
name match found
9419
name match found
9420
name match found
9421
name match found
9422
9423
name match found
9424
9425
name match found
9426
9427
9428
9429
name match found
9430
name match found
9431
9432
9433
name match found
9434
name match found
9435
9436
9437
name match found
9438
9439
9440
9441
9442
9443
name match found
9444
9445
9446
name match found
9447
9448
9449
name match found
9450
9451
name match found
9452
9453
9454
9455
9456
9457
9458
9459
9460
name match found
9461
9462
name match found
9463
name match found
9464
name

10042
name match found
10043
name match found
10044
name match found
10045
name match found
10046
name match found
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
name match found
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
name match found
10123
10124
name match found
10125
10126
name match found
10127
10128
10129
10130
10131
10132
name match found
10133
name match found
10134
name match found
10135
name match found
10136
name match found
10137
name match found
10138
name match found
10139
name match found
10140
10141
name match found
10142
name match found
10143
name match found
10144
name match found
10145
10146
10147
name ma

name match found
10658
name match found
10659
name match found
10660
name match found
10661
name match found
10662
10663
10664
10665
10666
10667
10668
10669
10670
10671
10672
name match found
10673
10674
10675
10676
10677
name match found
10678
name match found
10679
name match found
10680
name match found
10681
10682
10683
10684
name match found
10685
name match found
10686
name match found
10687
name match found
10688
10689
10690
10691
name match found
10692
name match found
10693
name match found
10694
10695
10696
name match found
10697
name match found
10698
10699
name match found
10700
10701
name match found
10702
name match found
10703
10704
10705
name match found
10706
10707
name match found
10708
10709
name match found
10710
10711
10712
name match found
10713
10714
name match found
10715
10716
10717
name match found
10718
name match found
10719
10720
10721
name match found
10722
10723
10724
name match found
10725
10726
10727
name match found
10728
name match found
10729
name ma

## Merge URL with charter data 

In [16]:
#merge based on combined name and address

charters_merge_CER = pd.merge(charters_nonduplicate_with_address, CER, how = 'left', left_on = ['address_name_combined'], right_on = ['address_name_combined']) 



In [17]:
CER_columns = list(CER.columns) #list of all columns labels in CER
CER_columns.remove('CER_URL') #remove CER_URL from the list 

In [18]:
charters_merge_CER = charters_merge_CER.drop(labels = CER_columns, axis = 1) #drop all columns in CER except CER_URL 

## Check URL matches

In [20]:
len(charters_nonduplicate_with_address)

10965

In [21]:
len(charters_merge_CER['NCESSCH']) #resulting merge has 6 more rows than original charters_nonduplicate

10971

In [22]:
#number of non-duplicate, non-NAN URLs in CER
non_null_or_duplicate_CER_URL = set(CER['CER_URL'].dropna())
len(non_null_or_duplicate_CER_URL)

4810

In [23]:
#number of non-duplicate, non-NAN URL matches
non_null_or_duplicate_vals = set(charters_merge_CER['CER_URL'].dropna())
len(non_null_or_duplicate_vals)

4189

In [32]:
charters_merge_CER[['SCHNAM12', 'CER_URL']].dropna()

,SCHNAM12,CER_URL
1,ayaprun elitnaurvik,http://www.yupik.org/#
2,ketchikan charter school,http://kcs.kgbsd.org/#
3,tongass school of arts and sciences charter sc...,http://www.tongassschool.org/#
4,aquarian charter school,http://www.aquariancharterschool.com/#
5,family partnership charter school,http://www.fpcs.net/#
8,winterberry school,http://www.winterberryschool.org/#
9,eagle academy charter school,http://www.eagleacademycharterschool.com/#
10,frontier charter school,http://www.frontierk12.org/#
11,highland tech high charter school,http://www.highlandtech.org/#
13,alaska native cultural charter school,http://www.asdk12.org/schools/anccs/pages/#


In [ ]:
#charters_merge_CER.to_csv('../../nowdata/backups/charters_full_250_CER_URLs.pkl')